# 🎬 ViralCutter - Cyclic Smooth Zoom Edition (Kaggle)

Versão especial com **YOLO Tracking + Cyclic Smooth Zoom** automático!

### ✨ Novidades v0.9
- 🔄 **Zoom Cíclico**: zoom in (3s) → hold (2s) → **SNAP BACK** (instantâneo) → repete
- 🎥 **Tracking Ultra Suave**: alpha 0.02 (câmera segue o rosto bem devagar)
- ⚡ **T4 Otimizado**: WhisperX com int8 quantization (50% menos VRAM)

### 🚀 Recursos
- ✅ Download de vídeos do YouTube
- ✅ Cortes automáticos com IA (Gemini)
- ✅ Legendas dinâmicas
- ✅ Upload automático para Google Drive
- ✅ Transcrição otimizada para GPUs T4

---

## 📦 Datasets Necessários

| Dataset | Descrição |
|---------|----------|
| `client-secret-json` | OAuth do Google Cloud |
| `cookie` | Cookies YouTube (opcional) |
| `credenciais-google` | API Key Gemini |
| `google-drive-credentials` | Credenciais salvas (gerado após 1ª autenticação) |

---

In [ ]:
#@title 🚀 VIRALCUTTER CYCLIC SMOOTH ZOOM (Kaggle + T4 Otimizado)
#@markdown ### ⚙️ Zoom Cíclico + T4 Optimization
#@markdown **Zoom:**
#@markdown - Zoom In (3s) → Hold (2s) → SNAP BACK → Hold (2s) → Repete
#@markdown
#@markdown **WhisperX T4 Otimizado:**
#@markdown - Modelo: `large-v2` (quantizado)
#@markdown - Compute: `int8_float16` (50% menos VRAM)
#@markdown - Batch: 8 (evita OOM)
#@markdown - Idioma: Português (padrão)

import os
import time
import threading
import json
import shutil
import subprocess
import sys
from IPython.display import clear_output

# ==============================================================================
# 1. SETUP DE BIBLIOTECAS (FIX AUTH)
# ==============================================================================
print("1️⃣ Ajustando bibliotecas de autenticação...")
try:
    subprocess.run(
        [sys.executable, "-m", "pip", "install", "google-auth-oauthlib==0.4.6", "google-api-python-client", "--force-reinstall", "--no-deps", "-q"],
        check=True
    )
    subprocess.run(
        [sys.executable, "-m", "pip", "install", "google-auth>=1.0.0", "requests-oauthlib>=0.7.0", "-q"], 
        check=True
    )
except Exception as e:
    print(f"⚠️ Aviso na instalação de libs: {e}")

from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

# ==============================================================================
# 2. INSTALAÇÃO DO SISTEMA + YOLO CYCLIC SMOOTH ZOOM + T4 OPTIMIZATION
# ==============================================================================
VENV_PYTHON = '/kaggle/working/ViralCutter/.venv/bin/python'
BRANCH = "smooth-zoom"  # Branch com Cyclic Smooth Zoom + T4 Optimization

if not os.path.exists(VENV_PYTHON):
    print(f"\n📦 Instalando ViralCutter + Cyclic Smooth Zoom + T4 Opt (branch: {BRANCH})...")
    
    if os.path.exists("/kaggle/working/ViralCutter"):
        shutil.rmtree("/kaggle/working/ViralCutter")
    
    # Clone da branch smooth-zoom
    subprocess.run(f"git clone -b {BRANCH} https://github.com/masterface77/ViralCutter.git /kaggle/working/ViralCutter", shell=True, check=True)
    os.chdir("/kaggle/working/ViralCutter")
    
    print("   - Instalando drivers...")
    subprocess.run("pip install uv -q", shell=True)
    subprocess.run("apt-get update -y -qq && apt-get install -y libcudnn8 ffmpeg xvfb -qq", shell=True)
    
    print("   - Criando ambiente virtual...")
    subprocess.run(["uv", "venv", ".venv"], check=True)
    
    print("   - Baixando pacotes de IA + Ultralytics (T4 Otimizado)...")
    cmds = [
        "uv pip install --python .venv faster-whisper",
        "uv pip install --python .venv git+https://github.com/m-bain/whisperx.git",
        "uv pip install --python .venv -r requirements.txt",
        "uv pip install --python .venv -U --pre 'yt-dlp[default]'",
        "uv pip install --python .venv pytubefix google-generativeai pandas onnxruntime-gpu",
        "uv pip install --python .venv transformers==4.46.3 accelerate>=0.26.0",
        "uv pip install --python .venv torch==2.3.1+cu121 torchvision==0.18.1+cu121 torchaudio==2.3.1+cu121 --index-url https://download.pytorch.org/whl/cu121",
        "uv pip install --python .venv 'numpy<2.0' setuptools==69.5.1",
        "uv pip install --python .venv insightface onnxruntime-gpu",
        # YOLO Cyclic Smooth Zoom
        "uv pip install --python .venv ultralytics",
        "uv pip uninstall --python .venv mediapipe protobuf flatbuffers",
        "uv pip install --python .venv 'mediapipe>=0.10.0' 'protobuf>=3.20,<5.0' 'flatbuffers>=2.0'"
    ]
    
    for cmd in cmds:
        subprocess.run(cmd, shell=True, check=True)
        
    print("✅ Instalação Concluída!")
    print("🔄 Cyclic Smooth Zoom + T4 Optimization ativados!")
    print("⚡ WhisperX: int8_float16, batch_size=8, large-v2")
else:
    os.chdir("/kaggle/working/ViralCutter")
    print("\n✅ Sistema já instalado.")
    # Atualizar para última versão
    subprocess.run("git pull origin smooth-zoom", shell=True)
    print("🔄 Atualizado para última versão!")

# ==============================================================================
# 3. PATCH NO DOWNLOADER
# ==============================================================================
print("\n🔧 Aplicando patch no sistema de download...")
download_script_content = r'''
import yt_dlp
import os
import glob
import shutil

def download(url, download_subs=False, quality="best", output_folder="VIRALS"):
    print(f"🚀 Iniciando download: {url}")
    title_temp = "video_temp"
    try:
        with yt_dlp.YoutubeDL({'quiet': True}) as ydl:
            info = ydl.extract_info(url, download=False)
            if info:
                t = info.get('title', 'video_temp')
                title_temp = "".join([c for c in t if c.isalnum() or c in " .-_"]).strip()
    except: pass

    final_path = os.path.join(output_folder, title_temp)
    os.makedirs(final_path, exist_ok=True)
    output_template = f"{final_path}/input.%(ext)s"

    ydl_opts = {
        'format': "bestvideo+bestaudio/best",
        'outtmpl': output_template,
        'noplaylist': True,
        'writesubtitles': False,
        'writeautomaticsub': False,
        'extractor_args': {'youtube': {'player_client': ['android', 'web']}}, 
        'quiet': False,
        'no_warnings': True,
        'merge_output_format': 'mp4'
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        
        mp4_files = glob.glob(os.path.join(final_path, "*.mp4"))
        if mp4_files:
            found_file = mp4_files[0]
            print(f"✅ Arquivo baixado: {found_file}")
            expected_input = os.path.join(final_path, "input.mp4")
            if found_file != expected_input:
                shutil.move(found_file, expected_input)
                return expected_input
            return found_file
        raise FileNotFoundError("Arquivo MP4 não gerado")
    except Exception as e:
        print(f"❌ Falha crítica no Download: {e}")
        raise e
'''
with open('/kaggle/working/ViralCutter/scripts/download_video.py', 'w') as f:
    f.write(download_script_content)

# ==============================================================================
# 4. AUTENTICAÇÃO OAUTH
# ==============================================================================
CLIENT_SECRET_FILE = None
SCOPES = ['https://www.googleapis.com/auth/drive.file']
API_SERVICE_NAME = 'drive'
API_VERSION = 'v3'

print("\n🔑 Procurando 'client_secret.json'...")
for root, dirs, files in os.walk('/kaggle/input'):
    for file in files:
        if 'client_secret' in file and file.endswith('.json'):
            CLIENT_SECRET_FILE = os.path.join(root, file)
            print(f"✅ Credencial encontrada: {CLIENT_SECRET_FILE}")
            break
    if CLIENT_SECRET_FILE: break

service = None
folder_id = None

if CLIENT_SECRET_FILE:
    print("\n🌐 INICIANDO LOGIN GOOGLE...")
    print("   ⚠️ Siga os passos abaixo com atenção:")
    print("   1. Clique no link que aparecerá abaixo.")
    print("   2. Faça login e autorize o acesso.")
    print("   3. Copie o código gerado pelo Google.")
    print("   4. COLE O CÓDIGO na caixa de entrada aqui no Kaggle e aperte Enter.\n")
    
    try:
        flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
        
        if hasattr(flow, 'run_console'):
            creds = flow.run_console()
        else:
            print("❌ AVISO CRÍTICO: O downgrade da biblioteca falhou.")
            print("   Tentando método alternativo via OOB...")
            flow.redirect_uri = 'urn:ietf:wg:oauth:2.0:oob'
            auth_url, _ = flow.authorization_url(prompt='consent')
            print(f"   Acesse este URL: {auth_url}")
            code = input("   Cole o código de autorização aqui: ")
            flow.fetch_token(code=code)
            creds = flow.credentials

        service = build(API_SERVICE_NAME, API_VERSION, credentials=creds)
        print("\n✅ Autenticação realizada com SUCESSO!")
        
        try:
            results = service.files().list(q="name='ViralCutter_SmoothZoom' and mimeType='application/vnd.google-apps.folder' and trashed=false", fields="files(id)").execute()
            items = results.get('files', [])
            if items:
                folder_id = items[0]['id']
                print(f"✅ Pasta 'ViralCutter_SmoothZoom' encontrada (ID: {folder_id})")
            else:
                file_metadata = {'name': 'ViralCutter_SmoothZoom', 'mimeType': 'application/vnd.google-apps.folder'}
                folder = service.files().create(body=file_metadata, fields='id').execute()
                folder_id = folder.get('id')
                print(f"✅ Pasta criada (ID: {folder_id})")
        except Exception as e:
            print(f"⚠️ Erro ao criar pasta: {e}")
    except Exception as e:
        print(f"❌ Erro no Login: {e}")
        print("   Nota: Se aparecer 'invalid_grant' ou 'redirect_uri_mismatch', suas credenciais podem não ser do tipo 'Desktop App'.")
else:
    print("❌ 'client_secret.json' não encontrado. O upload será desativado.")

# ==============================================================================
# 5. MONITOR DE UPLOAD OAUTH
# ==============================================================================
def monitor_oauth():
    if not service or not folder_id: 
        return
    
    print("\n👀 Monitor OAuth Ativo: Uploads usarão SEU espaço.")
    uploaded = set()
    
    while True:
        try:
            watch_path = '/kaggle/working/ViralCutter/VIRALS'
            if os.path.exists(watch_path):
                for r, d, f in os.walk(watch_path):
                    for file in f:
                        if not file.endswith('.mp4'):
                            continue
                        
                        ignore_patterns = [
                            'input.mp4', '_original_scale', '_subtitled', '_temp', 
                            'temp_', 'final_', '.part', 'processing'
                        ]
                        
                        if any(pattern in file.lower() for pattern in ignore_patterns):
                            continue
                        
                        path = os.path.join(r, file)
                        
                        if path not in uploaded:
                            try:
                                if os.path.getsize(path) > 1e6:
                                    s1 = os.path.getsize(path)
                                    time.sleep(5)
                                    if not os.path.exists(path):
                                        continue
                                    if os.path.getsize(path) != s1:
                                        continue
                                    
                                    print(f"\n💎 Novo Corte FINAL Detectado: {file}")
                                    print(f"   📁 Tamanho: {os.path.getsize(path) / 1e6:.2f} MB")
                                    print(f"   ☁️ Enviando para Google Drive (Conta Pessoal)...")
                                    
                                    try:
                                        meta = {'name': file, 'parents': [folder_id]}
                                        media = MediaFileUpload(path, resumable=True)
                                        service.files().create(body=meta, media_body=media).execute()
                                        uploaded.add(path)
                                        print("   ✅ Upload Completo!")
                                    except Exception as e_up:
                                        print(f"   ❌ Erro Upload: {e_up}")
                                        time.sleep(5)
                            except FileNotFoundError:
                                continue
            
            time.sleep(10)
        except Exception as e:
            print(f"⚠️ Erro no monitor: {e}")
            time.sleep(10)

threading.Thread(target=monitor_oauth, daemon=True).start()

# ==============================================================================
# 6. INICIAR INTERFACE
# ==============================================================================
print("\n🚀 INICIANDO VIRALCUTTER...")
print("⚡ T4 Otimizado: int8_float16, batch_size=8, large-v2")
print("⚠️ CLIQUE NO LINK PÚBLICO (gradio.live) ABAIXO:")
print("="*60)

subprocess.Popen(
    ['Xvfb', ':1', '-screen', '0', '2560x1440x8'],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)
time.sleep(2)

os.environ['DISPLAY'] = ':1.0'
os.environ['MPLBACKEND'] = 'Agg'
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

!/kaggle/working/ViralCutter/.venv/bin/python webui/app.py --colab --face-model yolo

## 📝 Créditos

**ViralCutter** por [Rafa.Godoy](https://github.com/rafaelGodoyEbert)

**Cyclic Smooth Zoom + T4 Optimization:**
- [Ultralytics YOLOv8](https://github.com/ultralytics/ultralytics) - Face tracking
- WhisperX com int8 quantization - Transcrição otimizada
- EMA alpha=0.02 - Camera suave

### ⚡ Configuração T4
```
WhisperX:  large-v2 | int8_float16 | batch=8 | lang=pt
Zoom:      1.0x ──(3s)──► 1.4x ──(2s)──► SNAP ──(2s)──► repete
```

---
`v0.9 Alpha + Cyclic Smooth Zoom + T4 Optimization`